# Краткое руководство по Policyscope
Policyscope позволяет оценивать новые рекомендательные политики на основе логов
существующей политики. Здесь мы на синтетических данных сравним политику B
с текущей политикой A.

## Алгоритмы и допущения
- **Replay** — усредняет отклики только по тем случаям, где действия A и B совпали. Требует значительного пересечения действий.
- **IPS** — взвешивает записи по отношению вероятностей \pi_B/\pi_A. Нужны корректные пропенсити и ненулевой шанс всех действий.
- **SNIPS** — нормализует веса IPS, снижая дисперсию.
- **DM** — строит модель отклика \hat{\mu}(x,a). Предполагается правильная спецификация модели.
- **DR** — объединяет DM и IPS и остаётся несмещённым, если верна хотя бы одна часть.

## Формат данных
Логи политики A должны содержать столбцы: `user_id`, `a_A`, `propensity_A`, `accept`, `cltv` и признаки `loyal`, `age_z`, `risk_z`, `income_z`.

## Генерация синтетики и сравнение политик

In [1]:

import pandas as pd
from policyscope.synthetic import SynthConfig, SyntheticRecommenderEnv
from policyscope.policies import make_policy
from policyscope.estimators import (
    value_on_policy, replay_value, prepare_piB_taken,
    ips_value, snips_value, dm_value, dr_value, train_mu_hat,
)


In [2]:

cfg = SynthConfig(n_users=5000, horizon_days=30, seed=42)
env = SyntheticRecommenderEnv(cfg)
X = env.sample_users()
policyA = make_policy('epsilon_greedy', epsilon=0.1, seed=0)
policyB = make_policy('softmax', tau=0.7, seed=1)
logsA = env.simulate_logs_A(policyA, X)
logsA.head()


,user_id,loyal,age,risk,income,region,age_z,risk_z,income_z,a_A,propensity_A,accept,cltv
0,0,0,35,0.501541,14575.147294,1,-0.416667,0.006164,-1.842693,2,0.925,1,240.092667
1,1,1,29,0.732395,26513.050864,2,-0.916667,0.929582,-0.646610,2,0.925,1,826.611167
2,2,0,45,0.188225,22144.013597,0,0.416667,-1.247099,-1.006583,1,0.925,0,189.567727
3,3,0,52,0.443791,38523.334086,4,1.000000,-0.224838,0.100297,1,0.925,0,305.824421
4,4,1,31,0.532530,24493.569649,3,-0.750000,0.130121,-0.804989,2,0.925,0,623.350392


In [3]:

V_A = value_on_policy(logsA, target='accept')
a_B = policyB.action_argmax(X)
V_replay = replay_value(logsA, a_B, target='accept')
piB_taken = prepare_piB_taken(logsA, policyB)
mu_accept = train_mu_hat(logsA, target='accept')
V_ips, ess_ips, clip_ips = ips_value(logsA, piB_taken, target='accept', weight_clip=20)
V_snips, ess_snips, clip_snips = snips_value(logsA, piB_taken, target='accept', weight_clip=20)
V_dm = dm_value(logsA, policyB, mu_accept, target='accept')
V_dr, ess_dr, clip_dr = dr_value(logsA, policyB, mu_accept, target='accept', weight_clip=20)
V_A_true = env.oracle_value(policyA, X, metric='accept')
V_B_true = env.oracle_value(policyB, X, metric='accept')
res = pd.DataFrame({'Оценка V(B)': [V_replay, V_ips, V_snips, V_dm, V_dr]}, index=['Replay','IPS','SNIPS','DM','DR'])
res['ATE'] = res['Оценка V(B)'] - V_A
display(res)
print(f'Истинный V(A) = {V_A_true:.3f}, V(B) = {V_B_true:.3f}, ATE = {V_B_true - V_A_true:.3f}')


,Оценка V(B),ATE
Replay,0.420945,-0.051055
IPS,0.461673,-0.010327
SNIPS,0.519345,0.047345
DM,0.488425,0.016425
DR,0.523130,0.051130


Истинный V(A) = 0.461, V(B) = 0.533, ATE = 0.072


### Ссылки
- Евгений Ян. [Counterfactual Evaluation for Recommendation Systems](https://eugeneyan.com/writing/offline-recsys/)
- Farajtabar et al. *More Robust Doubly Robust Off-policy Evaluation* (2022)